# Multiprocessing
Tutorial by Jonas Wilfert, Tobias Thummerer

## License

In [1]:
# Copyright (c) 2021 Tobias Thummerer, Lars Mikelsons, Josef Kircher, Johannes Stoljar, Jonas Wilfert
# Licensed under the MIT license. 
# See LICENSE (https://github.com/thummeto/FMI.jl/blob/main/LICENSE) file in the project root for details.

## Motivation
This Julia Package *FMI.jl* is motivated by the use of simulation models in Julia. Here the FMI specification is implemented. FMI (*Functional Mock-up Interface*) is a free standard ([fmi-standard.org](http://fmi-standard.org/)) that defines a container and an interface to exchange dynamic models using a combination of XML files, binaries and C code zipped into a single file. The user can thus use simulation models in the form of an FMU (*Functional Mock-up Units*). Besides loading the FMU, the user can also set values for parameters and states and simulate the FMU both as co-simulation and model exchange simulation.

## Introduction to the example
This example shows how to parallelize the computation of an FMU in FMI.jl. We can compute a batch of FMU-evaluations in parallel with different initial settings.
Parallelization can be achieved using multithreading or using multiprocessing. This example shows **multiprocessing**, check `multithreading.ipynb` for multithreading.
Advantage of multithreading is a lower communication overhead as well as lower RAM usage.
However in some cases multiprocessing can be faster as the garbage collector is not shared.


The model used is a one-dimensional spring pendulum with friction. The object-orientated structure of the *SpringFrictionPendulum1D* can be seen in the following graphic.

![svg](https://github.com/thummeto/FMI.jl/blob/main/docs/src/examples/pics/SpringFrictionPendulum1D.svg?raw=true)  


## Target group
The example is primarily intended for users who work in the field of simulations. The example wants to show how simple it is to use FMUs in Julia.


## Other formats
Besides, this [Jupyter Notebook](https://github.com/thummeto/FMI.jl/blob/examples/examples/multiprocessing.ipynb) there is also a [Julia file](https://github.com/thummeto/FMI.jl/blob/examples/examples/multiprocessing.jl) with the same name, which contains only the code cells and for the documentation there is a [Markdown file](https://github.com/thummeto/FMI.jl/blob/examples/examples/multiprocessing.md) corresponding to the notebook.  


## Getting started

### Installation prerequisites
|     | Description                       | Command                   | Alternative                                    |   
|:----|:----------------------------------|:--------------------------|:-----------------------------------------------|
| 1.  | Enter Package Manager via         | ]                         |                                                |
| 2.  | Install FMI via                   | add FMI                   | add " https://github.com/ThummeTo/FMI.jl "     |
| 3.  | Install FMIZoo via                | add FMIZoo                | add " https://github.com/ThummeTo/FMIZoo.jl "  |
| 4.  | Install FMICore via               | add FMICore               | add " https://github.com/ThummeTo/FMICore.jl " |
| 5.  | Install BenchmarkTools via        | add BenchmarkTools        |                                                |

## Code section



Adding your desired amount of processes:

In [2]:
using Distributed
n_procs = 2
addprocs(n_procs; exeflags=`--project=$(Base.active_project()) --threads=auto`, restrict=false)

2-element Vector{Int64}:
 2
 3

To run the example, the previously installed packages must be included. 

In [3]:
# imports
@everywhere using FMI
@everywhere using FMIZoo
@everywhere using BenchmarkTools

Checking that we workers have been correctly initialized:

In [4]:
workers()

@everywhere println("Hello World!")

# The following lines can be uncommented for more advanced informations about the subprocesses
# @everywhere println(pwd())
# @everywhere println(Base.active_project())
# @everywhere println(gethostname())
# @everywhere println(VERSION)
# @everywhere println(Threads.nthreads())

Hello World!
      From worker 2:	Hello World!
      From worker 3:	Hello World!


### Simulation setup

Next, the batch size and input values are defined.

In [5]:

# Best if batchSize is a multiple of the threads/cores
batchSize = 16

# Define an array of arrays randomly
input_values = collect(collect.(eachrow(rand(batchSize,2))))

16-element Vector{Vector{Float64}}:
 [0.1338617585355164, 0.44947497571089867]
 [0.008001957360047607, 0.9937485542708071]
 [0.20776949334971473, 0.6490825749531026]
 [0.40871964000115146, 0.5937682152425128]
 [0.40358041070764217, 0.17849443652751829]
 [0.7684229237645459, 0.46987805229505386]
 [0.7310722335509123, 0.059139703197925275]
 [0.3229732619906198, 0.027550543054871257]
 [0.9077630559649303, 0.8865471031841601]
 [0.9827476269494935, 0.5929815794209843]
 [0.799568853839973, 0.29747721684578465]
 [0.9882938809500592, 0.29407015570651884]
 [0.5432842493888508, 0.4784315239797283]
 [0.6758173044447395, 0.10160727436757244]
 [0.25157306026170567, 0.1880695834162025]
 [0.5613933983130321, 0.7015595974433939]

### Shared Module
For Distributed we need to embed the FMU into its own `module`. This prevents Distributed from trying to serialize and send the FMU over the network, as this can cause issues. This module needs to be made available on all processes using `@everywhere`.

In [6]:
@everywhere module SharedModule
    using FMIZoo
    using FMI

    t_start = 0.0
    t_step = 0.1
    t_stop = 10.0
    tspan = (t_start, t_stop)
    tData = collect(t_start:t_step:t_stop)

    model_fmu = FMIZoo.fmiLoad("SpringPendulum1D", "Dymola", "2022x")
end

┌ Info: fmi2Unzip(...): Successfully unzipped 153 files at `/tmp/fmijl_8hPF9Z/SpringPendulum1D`.
└ @ FMIImport /home/runner/.julia/packages/FMIImport/1Yngw/src/FMI2_ext.jl:90
[ Info: fmi2Unzip(...): Successfully unzipped 153 files at `/tmp/fmijl_tPX216/SpringPendulum1D`.
┌ Info: fmi2Load(...): FMU resources location is `file:////tmp/fmijl_8hPF9Z/SpringPendulum1D/resources`
└ @ FMIImport /home/runner/.julia/packages/FMIImport/1Yngw/src/FMI2_ext.jl:221
┌ Info: fmi2Load(...): FMU supports both CS and ME, using CS as default if nothing specified.
└ @ FMIImport /home/runner/.julia/packages/FMIImport/1Yngw/src/FMI2_ext.jl:224
[ Info: fmi2Load(...): FMU resources location is `file:////tmp/fmijl_tPX216/SpringPendulum1D/resources`
[ Info: fmi2Load(...): FMU supports both CS and ME, using CS as default if nothing specified.
[ Info: fmi2Unzip(...): Successfully unzipped 153 files at `/tmp/fmijl_cM2l6X/SpringPendulum1D`.
[ Info: fmi2Load(...): FMU resources location is `file:////tmp/fmijl_cM2l6X/S

We define a helper function to calculate the FMU and combine it into an Matrix.

In [7]:
@everywhere function runCalcFormatted(fmu, x0, recordValues=["mass.s", "mass.v"])
    data = fmiSimulateME(fmu, SharedModule.t_start, SharedModule.t_stop; recordValues=recordValues, saveat=SharedModule.tData, x0=x0, showProgress=false, dtmax=1e-4)
    return reduce(hcat, data.states.u)
end

Running a single evaluation is pretty quick, therefore the speed can be better tested with BenchmarkTools.

In [8]:
@benchmark data = runCalcFormatted(SharedModule.model_fmu, rand(2))

BenchmarkTools.Trial: 12 samples with 1 evaluation.
 Range (min … max):  417.415 ms … 455.643 ms  ┊ GC (min … max): 2.29% … 4.45%
 Time  (median):     435.929 ms               ┊ GC (median):    4.38%
 Time  (mean ± σ):   437.513 ms ±  11.672 ms  ┊ GC (mean ± σ):  3.93% ± 0.99%

  ▁       ▁       ▁         █  █      ▁       ▁       ▁ ▁     ▁  
  █▁▁▁▁▁▁▁█▁▁▁▁▁▁▁█▁▁▁▁▁▁▁▁▁█▁▁█▁▁▁▁▁▁█▁▁▁▁▁▁▁█▁▁▁▁▁▁▁█▁█▁▁▁▁▁█ ▁
  417 ms           Histogram: frequency by time          456 ms <

 Memory estimate: 128.48 MiB, allocs estimate: 1802004.

### Single Threaded Batch Execution
To compute a batch we can collect multiple evaluations. In a single threaded context we can use the same FMU for every call.

In [9]:
println("Single Threaded")
@benchmark collect(runCalcFormatted(SharedModule.model_fmu, i) for i in input_values)

Single Threaded


BenchmarkTools.Trial: 1 sample with 1 evaluation.
 Single result which took 6.814 s (4.19% GC) to evaluate,
 with a memory estimate of 2.01 GiB, over 28832052 allocations.

### Multithreaded Batch Execution
In a multithreaded context we have to provide each thread it's own fmu, as they are not thread safe.
To spread the execution of a function to multiple processes, the function `pmap` can be used.

In [10]:
println("Multi Threaded")
@benchmark pmap(i -> runCalcFormatted(SharedModule.model_fmu, i), input_values)

Multi Threaded


BenchmarkTools.Trial: 2 samples with 1 evaluation.
 Range (min … max):  4.331 s …   4.437 s  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     4.384 s              ┊ GC (median):    0.00%
 Time  (mean ± σ):   4.384 s ± 75.034 ms  ┊ GC (mean ± σ):  0.00% ± 0.00%

  █                                                       █  
  █▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁█ ▁
  4.33 s         Histogram: frequency by time        4.44 s <

 Memory estimate: 82.00 KiB, allocs estimate: 1222.

As you can see, there is a significant speed-up in the median execution time. But: The speed-up is often much smaller than `n_procs` (or the number of physical cores of your CPU), this has different reasons. For a rule of thumb, the speed-up should be around `n/2` on a `n`-core-processor with `n` Julia processes.

### Unload FMU

After calculating the data, the FMU is unloaded and all unpacked data on disc is removed.

In [11]:
@everywhere fmiUnload(SharedModule.model_fmu)

### Summary

In this tutorial it is shown how multi processing with `Distributed.jl` can be used to improve the performance for calculating a Batch of FMUs.